In [1]:
import pandas as pd
import pandas_ta as ta
from enum import Enum
from tvDatafeed import TvDatafeed ,Interval
import numpy as np
from backtesting.lib import resample_apply
from sklearn.preprocessing import StandardScaler,MinMaxScaler
import matplotlib.pyplot as plt
from backtesting import Backtest,Strategy
import seaborn as sns
from bs4 import BeautifulSoup
import requests
from bokeh.io import export_png
import os
from time import time
import warnings
warnings.filterwarnings("ignore")

C:\Users\Cuneyt\anaconda3\Lib\site-packages\backtesting\_plotting.py:50: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support (e.g. PyCharm, Spyder IDE). Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


Loading BokehJS ...

In [2]:
class Fields(Enum):
    SYMBOL = "Symbol"
    OPEN = "Open"
    CLOSE = "Close"
    HIGH = "High"
    LOW = "Low"
    VOLUME = "Volume"

In [3]:
def rename_df(df):
    """
    Standardizing the names"""
    df.columns = [Fields.SYMBOL.value, Fields.OPEN.value,Fields.HIGH.value,Fields.LOW.value,Fields.CLOSE.value,Fields.VOLUME.value]
    return df

In [4]:
def pvt2(data):
    hlc3 = (data[Fields.HIGH.value]+data[Fields.LOW.value]+data[Fields.CLOSE.value])/3
    return hlc3.to_numpy()
def pvt1(data):
    return data[Fields.CLOSE.value].to_numpy()
def will_r(data,period): 
    will_r = ta.willr(data[Fields.HIGH.value],low=data[Fields.LOW.value],close=data[Fields.CLOSE.value],length=period)
    return will_r.to_numpy()


In [5]:
def get_data(stock_key:str, exchange:str,bars:int):
    tv = TvDatafeed()
    stock_key = stock_key
    exchange =exchange
    bars = bars
    timeframe =Interval.in_1_hour
    df = tv.get_hist(symbol=stock_key ,exchange=exchange, interval=timeframe,n_bars=bars)
    df=rename_df(df)
    return df
def find_stocks_from_KAP():
    """
    Sadece Yıldız ve Ana Pazardaki Hisseleri çeker.
    """
    url= "https://www.kap.org.tr/tr/Pazarlar"
    r = requests.get(url).text
    soup=BeautifulSoup(r)
    yildiz_pazar_tablo = soup.find_all("div", {"class": "column-type7 wmargin"})[0]
    ana_pazar_tablo = soup.find_all("div", {"class": "column-type7 wmargin"})[1]
    hisseler = []
    stock_html = yildiz_pazar_tablo.find_all("div" ,{"class":"comp-cell _02 vtable"})
    for row in stock_html:
        for data in row.find_all("a"):
            hisseler.append(data.text)
    stock_html = ana_pazar_tablo.find_all("div" ,{"class":"comp-cell _02 vtable"})
    for row in stock_html:
        for data in row.find_all("a"):
            hisseler.append(data.text)
    return hisseler

In [6]:
class PivotStrategy(Strategy):
    willr_period = 3
    willr_up_level = -20
    willr_down_level = -70

    def init(self):      
        self.pvt1 = resample_apply("D",pvt1,self.data.df)
        self.pvt2 = resample_apply("D",pvt2,self.data.df)
        self.will_r = self.I(will_r, self.data.df, period=self.willr_period)
    def next(self):
        if self.data.Close>max(self.pvt1,self.pvt2) and self.will_r > self.willr_up_level :
            self.buy()
        elif self.data.Close<min(self.pvt1,self.pvt2) and self.will_r < self.willr_down_level :
            self.position.close()


In [7]:
#hisseler = "AKBNK,ODAS"
#stock_list = hisseler.split(",")
stock_list = find_stocks_from_KAP()
starting_cash= 100000

In [8]:
result_df=pd.DataFrame()
directory = "hisseler/"
parent_dir = "C:/Users/Cuneyt/Python Tech Ind/"
path = os.path.join(parent_dir, directory) 

for stock in stock_list:
    start_time = time()    
    try:
        df = get_data(stock,"BIST",2400) ##2200 yaklaşık sene başını işaret ediyor. 
    except:
        print("HISSE yok")
    #### Başlangıçtaki lot miktarını bulup bitişteki margini hesaplatıyoruz
    start_price = df["Open"].head(1).to_list()[0]
    start_lot=starting_cash/start_price
    end_price= df["Close"].tail(1).to_list()[0]
    end_margin = end_price*start_lot
    #####
    bt = Backtest(df,PivotStrategy,cash = starting_cash)
    stats = bt.run()

    new_row= {
        "stock":[stock],
        "Start Date":[stats[0]],
        "End Date":[stats[1]],
        "Start Equity(TL)":[starting_cash],
        "Final Equity(TL)":[round(stats[4])],
        "Peak Equity(TL)":[round(stats[5])],
        "Return %":[round(stats[6],2)],
        "Buy&Hold Return%":[round(stats[7],2)],
        "Max. Drawdown":[round(stats[14],2)],
        "# Trades":[stats[17]],
        "Win Rate %":[round(stats[18],1)],
        "Avg. Trade %":[round(stats[21],1)]}
    new_df = pd.DataFrame(new_row,index=[stock])
    result_df = pd.concat([result_df,new_df],axis=0)
    bt.plot(plot_volume=False,filename=path+stock,open_browser=False)
    end_time =time()
    print(f"{stock} TAMAMLANDI. Süre:{end_time-start_time}")

you are using nologin method, data you access may be limited


BINHO TAMAMLANDI. Süre:2.1801035404205322


A1CAP TAMAMLANDI. Süre:2.6200666427612305


ADGYO TAMAMLANDI. Süre:2.393644332885742


AFYON TAMAMLANDI. Süre:3.4363675117492676


...

YGYO TAMAMLANDI. Süre:4.535703897476196


YYAPI TAMAMLANDI. Süre:2.8933613300323486


YESIL TAMAMLANDI. Süre:3.1221702098846436


YKSLN TAMAMLANDI. Süre:3.180996894836426


YUNSA TAMAMLANDI. Süre:3.3446896076202393
ZEDUR TAMAMLANDI. Süre:2.957221031188965


In [9]:
result_df.reset_index(inplace=True)
result_df.drop("index",axis=1,inplace=True)
result_df.to_excel(f"{path}PHL2 Rapor.xlsx",sheet_name="PHL2")